In [1]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV에서는 기본적으로 BGR로 이미지를 로드합니다. RGB로 변환합니다.
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


In [4]:
def calculate_mean_std(loader):
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0)  # 배치 크기
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    mean /= len(loader.dataset)
    std /= len(loader.dataset)
    return mean, std

In [7]:
image_dir = '폴더이름'
image_paths = [os.path.join(image_dir, x) for x in os.listdir(image_dir)]
labels = [0 if 'class0' in x else 1 for x in image_paths]  # 예시로 레이블을 생성합니다. 실제 사용 시 적절히 조정해야 합니다.

In [ ]:
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [ ]:
train_dataset = CustomImageDataset(train_paths, train_labels)
test_dataset = CustomImageDataset(test_paths, test_labels)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

In [ ]:
# 평균과 표준편차 계산
mean, std = calculate_mean_std(train_loader)

# 정규화를 포함한 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
# 변환 적용
train_dataset = CustomImageDataset(train_paths, train_labels, transform=transform)
test_dataset = CustomImageDataset(test_paths, test_labels, transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)